In [27]:
import numpy as np
import gzip, pickle
import pandas as pd

patients = pd.read_csv('internacoes_charlson_zero.csv.gz', compression='gzip', \
                       nrows=None, usecols=['target'])
target = patients['target'].values
patients.shape

(48907, 1)

In [28]:
with gzip.open("data_5k_unigram.npy.gz", "rb") as wfp:   #Pickling
    unigram = pickle.load(wfp)
    wfp.close()
    
with gzip.open("data_5k_bigram.npy.gz", "rb") as wfp:   #Pickling
    bigram = pickle.load(wfp)
    wfp.close()
    
with gzip.open("data_5k_trigram.npy.gz", "rb") as wfp:   #Pickling
    trigram = pickle.load(wfp)
    wfp.close()
    
with gzip.open("data_10k_multigram.npy.gz", "rb") as wfp:   #Pickling
    multigram = pickle.load(wfp)
    wfp.close()
        
print (unigram.shape, bigram.shape, trigram.shape, multigram.shape)

(48907, 5000) (48907, 5000) (48907, 5000) (48907, 10000)


In [29]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [30]:
from collections import OrderedDict
grams = OrderedDict({ 
    'unigram': unigram,
    'bigram': bigram,
    'trigram': trigram,
    'multigram': multigram,
})

In [31]:
for size in range(250,5001,250):
    
    results = {}
    
    for name, gram in grams.items():
        
        model = RandomForestRegressor(n_jobs=5)

        target1 = np.asarray(target[:5000])
        data = gram[:len(target1),:250].todense()

        scores = cross_val_score(model, data, target1, cv=10, scoring='neg_mean_absolute_error', n_jobs=10)
        results[name] = np.abs(round(np.mean(scores),2))
    
    print(size, results['unigram'], results['bigram'], results['trigram'], results['multigram'])

250     2.19    2.44    2.44    2.39
500     1.94    2.35    2.34    2.34
750     1.86    2.31    2.17    2.26
1000    1.65    2.11    1.82    2.16
1250    1.58    1.92    1.74    2.08
1500    1.54    1.7     1.72    1.93
1750    1.52    1.64    1.69    1.88
2000    1.49    1.62    1.64    1.71
2250    1.5     1.6     1.62    1.67
2500    1.5     1.58    1.61    1.59
2750    1.47    1.56    1.6     1.56
3000    1.46    1.54    1.59    1.56
3250    1.45    1.53    1.59    1.54
3500    1.42    1.53    1.58    1.53
3750    1.42    1.51    1.58    1.51
4000    1.42    1.51    1.58    1.5
4250    1.42    1.5     1.57    1.49
4500    1.42    1.5     1.57    1.49
4750    1.41    1.5     1.57    1.48
5000    1.41    1.5     1.57    1.48
